# Build a Basic Bot

In [39]:
import time
import random

In [40]:
def answer(user_input):
    """Take the user's input and respond in kind."""
    resp = f"{user_input} back at ya!"
    time.sleep(0.5)
    return resp
answer("Heya")

'Heya back at ya!'

In [41]:
def user_speaks(user_input, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    print(f"{user_format} {user_input}")
    resp = answer(user_input)
    return f"{bot_format} {resp}"

user_speaks("Yoyoyo!")


USER: Yoyoyo!


'BOT: Yoyoyo! back at ya!'

## Basic Chat

Introduces some stock responses for exact question matches. Adds in some limited variation.

### Stock Answers

In [42]:
food = "ice cream"
weather = "overcast"
series = "The Rings of Power"
stock_qs = {
    "what's your favourite food?": f"I really like {food}.",
    "how's the weather today?": f"It's looking {weather} right now.",
    "what are you currently watching?": f"{series}, it's really great!"

}

In [43]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = stock_qs[user_input]
    else:
        resp = answer(user_input)
    return resp


In [44]:
def user_speaks(user_input, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    print(f"{user_format} {user_input}")
    resp = stock_answer(user_input)
    return f"{bot_format} {resp}"

In [45]:
user_speaks("what are you currently watching?")

USER: what are you currently watching?


"BOT: The Rings of Power, it's really great!"

In [46]:
user_speaks("That's rubbish!")

USER: That's rubbish!


"BOT: That's rubbish! back at ya!"

### Variation

In [47]:
stock_qs = {
    "what's your favourite food?": [
        f"I really like {food}.",
        f"{food} is my favourite.",
        f"I'm addicted to {food}."
        ],
    "how's the weather today?": [
        f"It's looking {weather} right now.",
        f"Today it's mainly {weather}."
        ],
    "what are you currently watching?": [
        f"{series}, it's really great!",
        f"I just can't get enough of {series} right now.",
        f"Way too much {series}."
        ]

}

In [48]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = random.choice(stock_qs[user_input])
    else:
        resp = answer(user_input)
    return resp

In [51]:
print(user_speaks("what are you currently watching?"))
print(user_speaks("what are you currently watching?"))
print(user_speaks("what are you currently watching?"))


USER: what are you currently watching?
BOT: Way too much The Rings of Power.
USER: what are you currently watching?
BOT: Way too much The Rings of Power.
USER: what are you currently watching?
BOT: The Rings of Power, it's really great!


## Asking Qs

In [59]:
is_question = {
    True: [
        "Dunno!",
        "Ummm...",
        "Don't have a scooby!"
    ],
    False: [
        "How are you today :) ?",
        "Did you remember to feed the dogs?",
        "Where did you go on hols?"
    ]
}

In [57]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = random.choice(stock_qs[user_input])
    else:
        # Replace the stock answer with the q check
        resp = random.choice(is_question[user_input.endswith("?")])
    return resp

In [61]:
print(user_speaks("what are you currently watching?"))
print(user_speaks("How are you feeling?"))
print(user_speaks("How are you feeling?"))
print(user_speaks("Feeling great."))
print(user_speaks("I love cake."))

USER: what are you currently watching?
BOT: The Rings of Power, it's really great!
USER: How are you feeling?
BOT: Ummm...
USER: How are you feeling?
BOT: Dunno!
USER: Feeling great.
BOT: Where did you go on hols?
USER: I love cake.
BOT: Did you remember to feed the dogs?
